## Crawl 1111

In [5]:
from bs4 import BeautifulSoup
import requests
import pymysql
import re

In [6]:
def getPageContent(url):
    resp = requests.get(url)
    soup = BeautifulSoup(resp.text, 'html.parser')
    return soup

In [7]:
conn = pymysql.connect(host='localhost', port=3306, user='root', password='1234', db='jobs', use_unicode=True, charset="utf8mb4", autocommit=True)
cur = conn.cursor()
insert_1111_template = "insert into job1(site, company, title, location, skills) values(1111, %s, %s, %s, %s)"

keyword = '%E8%BB%9F%E9%AB%94%E8%A8%AD%E8%A8%88%E5%B7%A5%E7%A8%8B%E5%B8%AB' # 軟體設計工程師
url_template = 'https://www.1111.com.tw/job-bank/job-index.asp?si=1&ss=s&ks=' + keyword + '&pt=1&ps=100&page={}'
page = 0

while True:
    page += 1
    job_links = []
    job_soup = getPageContent(url_template.format(page)) 
    print('page: ' + str(page)) # 列印出本頁面號
    anchors = job_soup.select('.JobsCss a')
    
    if len(anchors) > 0: # 若該頁面有職缺連結才爬
        for link in anchors:
            job_links.append('https:' + link['href'])
        
        all_jobs_in_page = [] # 【每頁所有職缺】初始化
        
        for job_link in job_links:
            job_detail_soup = getPageContent(job_link)
            
            job = {} # 【職缺】初始化
            
            # 公司
            company = job_detail_soup.select_one('.ellipsis')
            if company is not None:
                job['company'] = company.text
            else:
                job['company'] = ''
                
            # 職位
            h1 = job_detail_soup.select_one('h1')
            if h1 is not None:
                job['title'] = str(h1.find('span').previousSibling)
            else:
                job['title'] = ''
            
            # 工作地點
            listTitle = job_detail_soup.select('.listTitle') # 避免有些職缺Title出現'工作地點'，所以一律從.listTitle爬
            loc_title = [title for title in listTitle if title.text == '工作地點：']
            if len(loc_title) > 0:
                job['location'] = loc_title[0].findNext('div').text.replace('地圖','').replace(' ','')
            else:
                job['location'] = ''
            
            # 電腦專長
            listTitle = job_detail_soup.select('.listTitle') # 避免有些職缺Title出現'電腦專長'，所以一律從.listTitle爬
            skill_title = [title for title in listTitle if title.text == '電腦專長：']
            if len(skill_title) > 0:
                job['skills'] = skill_title[0].findNext('div').text.replace('Java Script', 'JavaScript').replace('、',' ')
            else:
                job['skills'] = ''
            
            all_jobs_in_page.append(job) # 把【職缺】放入到【每頁所有職缺】陣列
            
        jobs_tuple_arr = [(job['company'], job['title'], job['location'], job['skills'], ) for job in all_jobs_in_page]
        cur.executemany(insert_1111_template, jobs_tuple_arr) # 把本頁面所有職缺寫入DB
        
    else:
        break # 若該頁面沒有職缺連結就中斷

cur.close
conn.close

page: 1
page: 2
page: 3
page: 4
page: 5
page: 6
page: 7
page: 8
page: 9
page: 10
page: 11
page: 12
page: 13
page: 14
page: 15
page: 16
page: 17
page: 18
page: 19
page: 20
page: 21
page: 22
page: 23
page: 24
page: 25
page: 26
page: 27
page: 28
page: 29
page: 30
page: 31
page: 32
page: 33
page: 34
page: 35
page: 36
page: 37
page: 38
page: 39
page: 40


<bound method Connection.close of <pymysql.connections.Connection object at 0x082B8570>>